In [1]:
## 1. 라이브러리 읽어 들이기 ##

# 텐서플로 라이브러리
import tensorflow as tf
# tflearn 라이브러리
import tflearn

# 레이어 생성 등 학습에 필요한 라이브러리 읽어 들이기
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

import os
import numpy as np
from PIL import Image

In [2]:
## 이미지 데이터 처리하기

# 학습 전용 이미지 파일을 저장하고 있는 디렉터리
train_dirs = ['pos', 'neg']

# 학습 데이터를 저장할 배열 준비하기
trainX = [] # 이미지 픽셀값
trainY = [] # 정답 데이터

for i, d in enumerate(train_dirs):
    # 파일 이름 추출하기
    files = os.listdir('./data/pict/' + d)
    for f in files:
        # 이미지 읽어 들이기
        image = Image.open('./data/pict/' + d + '/' + f, 'r')
        # 그레이스케일로 변환하기
        gray_image = image.convert('L')
        # 이미지 파일을 픽셀값으로 변환하기
        gray_image_px = np.array(gray_image)
        gray_image_flatten = gray_image_px.flatten().astype(np.float32)/255.0
        trainX.append(gray_image_flatten)
        
        # 정답 데이터를 one_hot 형식으로 변환하기
        tmp = np.zeros(2)
        tmp[i] = 1
        trainY.append(tmp)

# numpy 배열로 변환하기
trainX = np.asarray(trainX)
trainY = np.asarray(trainY)

In [3]:
# 0번째 이미지 픽셀 값과 정답 데이터 출력하기
print(trainX[0])
print(trainY[0])

[ 0.26666668  0.42745098  0.45882353 ...,  0.27450982  0.27843139
  0.29803923]
[ 1.  0.]


In [4]:
# 0번째 이미지 픽셀 값과 정답 데이터의 크기 출력하기
print(len(trainX[0]))
print(len(trainY[0]))

1024
2


In [5]:
# 이미지 픽셀 데이터를 2차원으로 변환하기
trainX = trainX.reshape([-1, 32, 32, 1])

In [6]:
# 0번째 이미지 픽셀값 출력하기
print(trainX[0])

[[[ 0.26666668]
  [ 0.42745098]
  [ 0.45882353]
  ..., 
  [ 0.47450981]
  [ 0.51372552]
  [ 0.52941179]]

 [[ 0.35686275]
  [ 0.75686276]
  [ 0.82745099]
  ..., 
  [ 0.47450981]
  [ 0.48627451]
  [ 0.47058824]]

 [[ 0.34901962]
  [ 0.6156863 ]
  [ 0.80392158]
  ..., 
  [ 0.4509804 ]
  [ 0.45882353]
  [ 0.4627451 ]]

 ..., 
 [[ 0.03137255]
  [ 0.02745098]
  [ 0.00392157]
  ..., 
  [ 0.29803923]
  [ 0.27058825]
  [ 0.30980393]]

 [[ 0.01568628]
  [ 0.04313726]
  [ 0.04705882]
  ..., 
  [ 0.33725491]
  [ 0.32941177]
  [ 0.32156864]]

 [[ 0.02352941]
  [ 0.04313726]
  [ 0.07450981]
  ..., 
  [ 0.27450982]
  [ 0.27843139]
  [ 0.29803923]]]


In [7]:
# 0번째 이미지 픽셀값의 크기 출력하기
print(len(trainX[0]))

32


In [8]:
## 3. 신경망 만들기

## 초기화
tf.reset_default_graph()

## 입력 레이어 만들기
net = input_data(shape=[None, 32, 32, 1])

## 중간 레이어 만들기
# 합성곱 레이어 만들기
net = conv_2d(net, 32, 5, activation='relu')
# 풀링 레이어 만들기
net = max_pool_2d(net, 2)
# 합성곱 레이어 만들기
net = conv_2d(net, 64, 5, activation='relu')
# 풀링 레이어 만들기
net = max_pool_2d(net, 2)
# 전결합 레이어 만들기
net = fully_connected(net, 128, activation='relu')
net = dropout(net, 0.5)

## 출력 레이어 만들기
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.5, loss='categorical_crossentropy')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
## 4. 모델 만들기(학습) ##
# 학습 실행하기
model = tflearn.DNN(net)
model.fit(trainX, trainY, n_epoch=20, batch_size=32, validation_set=0.2, show_metric=True)

Training Step: 259  | total loss: 0.68022 | time: 1.276s
| SGD | epoch: 020 | loss: 0.68022 - acc: 0.5741 -- iter: 384/387
Training Step: 260  | total loss: 0.68371 | time: 2.376s
| SGD | epoch: 020 | loss: 0.68371 - acc: 0.5667 | val_loss: 0.60412 - val_acc: 0.7732 -- iter: 387/387
--
